# 搜索增强生成（RAG，Retrieval-Augmented Generation）

## 💡 这节课会带给你


1. 如何用你的垂域数据补充 LLM 的能力
1. 如何构建你的垂域（向量）知识库
1. 搭建一套完整 RAG 系统 Pipeline

开始上课！


## 一、澄清一个概念

RAG **不要** 参考下面这张图！！！

<img src="rag-paper.png" style="margin-left: 0px" width="600px">

这张图源自一个[研究工作](https://arxiv.org/pdf/2005.11401.pdf)
- 此论文第一次提出 RAG 这个叫法
- 在研究中，作者尝试将检索和生成做在一个模型体系中

**但是，实际生产中，RAG 不是这么做的！！！**

## 二、什么是检索增强的生成模型（RAG）


### 2.1、LLM 固有的局限性

1. LLM 的知识不是实时的
2. LLM 可能不知道你私有的领域/业务知识

<img src="gpt-llama2.png" style="margin-left: 0px" width="600px">


### 2.2、检索增强生成

天然能想到的，我们自己有产品知识库，有服务手册这些垂直领域的信息，能不能让大模型学会这些垂直领域的信息。
我们能想象到的方法有两种：

1. 重新训练大模型，把这些垂直领域的数据喂给大模型，让大模型从中学习, 这是微调
2. 给大模型添加个外挂的知识库，我们让大模型和这个知识库结合着去给用户回答问题

<div class="alert alert-success">
<b>类比：</b>
    <li>你可以把这个过程想象成开卷考试。让 LLM 先翻书，再回答问题。这个过程模型本身是不学会知识的。</li>
    <li>微调就是闭卷考试，你的先把所有的知识都学会，才能去回答问题。</li>
</div>

RAG（Retrieval Augmented Generation）顾名思义，通过**检索**的方法来增强**生成模型**的能力。

<video src="RAG.mp4" controls="controls" width=800px style="margin-left: 0px"></video>


## 三、RAG 系统的基本搭建流程

搭建过程：

1. 文档加载，并按一定条件**切割**成片段
2. 将切割的文本片段灌入**检索引擎**
3. 封装**检索接口**：能从文档里搜索出相关的文档片段
4. 构建**调用流程**：Query -> 检索 -> Prompt -> LLM -> 回复



### 3.1、文档的加载与切割


In [2]:
!pip install --upgrade openai

  Using cached openai-1.34.0-py3-none-any.whl.metadata (21 kB)
Using cached openai-1.34.0-py3-none-any.whl (325 kB)


In [3]:
# 安装 pdf 解析库
!pip install pdfminer.six

  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 16.4 MB/s eta 0:00:0000:0100:01


In [1]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer

In [2]:
def extract_text_from_pdf(filename, page_numbers=None, min_line_length=1):
    '''从 PDF 文件中（按指定页码）提取文字'''
    paragraphs = []
    buffer = ''
    full_text = ''
    # 提取全部文本
    for i, page_layout in enumerate(extract_pages(filename)):
        # 如果指定了页码范围，跳过范围外的页
        if page_numbers is not None and i not in page_numbers:
            continue
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                full_text += element.get_text() + '\n'
                
    # 按空行分隔，将文本重新组织成段落
    lines = full_text.split('\n')
    for text in lines:
        if len(text) >= min_line_length:
            buffer += (' '+text) if not text.endswith('-') else text.strip('-')
        elif buffer:
            paragraphs.append(buffer)
            buffer = ''
    if buffer:
        paragraphs.append(buffer)
    return paragraphs

In [3]:
paragraphs = extract_text_from_pdf("llama2.pdf", min_line_length=10)

In [4]:
for para in paragraphs[:3]:
    print(para+"\n")

 Llama 2: Open Foundation and Fine-Tuned Chat Models

 Hugo Touvron∗ Louis Martin† Kevin Stone† Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra Prajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen Guillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller Cynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou Hakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev Punit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich Yinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra Igor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang Ross Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang Angela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic Sergey Edu

## 3.2、检索引擎


这里我们使用先进的开源搜索引擎 ElasticSearch，它可以实现各种场景下的搜索功能。

官方地址：https://www.elastic.co/cn/elasticsearch(有兴趣的同学可以了解)

### 安装 ES 服务器

安装教程地址 https://www.elastic.co/guide/en/elasticsearch/reference/current/install-elasticsearch.html 。
（可以使用 cursor 参考学习）

安装后，可以通过不同系统的服务状态监测指令查看 ES 运行状态，这里我的 centos 指令为 `service elasticsearch status`

### 安装 ES 客户端 

In [8]:
!pip install elasticsearch7  

  Using cached elasticsearch7-7.17.9-py2.py3-none-any.whl.metadata (5.7 kB)
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
Using cached elasticsearch7-7.17.9-py2.py3-none-any.whl (386 kB)
Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1


### 安装NLTK（文本处理方法库）

In [9]:
!pip install nltk

  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.7 MB/s eta 0:00:00
Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.2/776.2 kB 13.8 MB/s eta 0:00:00a 0:00:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)


In [5]:
from elasticsearch7 import Elasticsearch, helpers
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import re

import warnings
warnings.simplefilter("ignore")  # 屏蔽 ES 的一些Warnings

# 下载分词器和停用词库
nltk.download('punkt')  # 英文切词、词根、切句等方法
nltk.download('stopwords')  # 英文停用词库

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
def to_keywords(input_string):
    '''（英文）文本只保留关键字'''
    # 使用正则表达式替换所有非字母数字的字符为空格
    no_symbols = re.sub(r'[^a-zA-Z0-9\s]', ' ', input_string)
    word_tokens = word_tokenize(no_symbols)
    # 加载停用词表
    stop_words = set(stopwords.words('english'))
    ps = PorterStemmer()
    # 去停用词，取词根
    filtered_sentence = [ps.stem(w)
                         for w in word_tokens if not w.lower() in stop_words]
    return ' '.join(filtered_sentence)

In [7]:
to_keywords('how many parameters does llama 2 have?')

'mani paramet llama 2'

<div class="alert alert-info">
此处 to_keywords 为针对英文的实现，针对中文的实现请参考 chinese_utils.py
</div>

将文本灌入检索引擎


In [8]:
# 1. 创建Elasticsearch连接
es = Elasticsearch(
    hosts=['http://localhost:9200'],  # 服务地址与端口
    # http_auth=("elastic", "FKaB1Jpz0Rlw0l6G"),  # 用户名，密码
)

# 2. 定义索引名称
index_name = "teacher_demo_index_tmp"

# 3. 如果索引已存在，删除它（仅供演示，实际应用时不需要这步）
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

# 4. 创建索引
es.indices.create(index=index_name)

# 5. 灌库指令，构建索引
actions = [
    {
        "_index": index_name,
        "_source": {
            "keywords": to_keywords(para),
            "text": para
        }
    }
    for para in paragraphs
]

# 6. 文本灌库
helpers.bulk(es, actions)

(983, [])

实现关键字检索


In [9]:
def search(query_string, top_n=3):
    # ES 的查询语言
    search_query = {
        "match": {
            "keywords": to_keywords(query_string)
        }
    }
    res = es.search(index=index_name, query=search_query, size=top_n)
    return [hit["_source"]["text"] for hit in res["hits"]["hits"]]

In [10]:
results = search("how many parameters does llama 2 have?", 2)
for r in results:
    print(r+"\n")

 Llama 2 comes in a range of parameter sizes—7B, 13B, and 70B—as well as pretrained and fine-tuned variations.

 1. Llama 2, an updated version of Llama 1, trained on a new mix of publicly available data. We also increased the size of the pretraining corpus by 40%, doubled the context length of the model, and adopted grouped-query attention (Ainslie et al., 2023). We are releasing variants of Llama 2 with 7B, 13B, and 70B parameters. We have also trained 34B variants, which we report on in this paper but are not releasing.§



### 3.3、LLM 接口封装


In [11]:
from openai import OpenAI
import os
# 加载环境变量
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # 读取本地 .env 文件，里面定义了 OPENAI_API_KEY

client = OpenAI()

In [12]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''封装 openai 接口'''
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,  # 模型输出的随机性，0 表示随机性最小
    )
    return response.choices[0].message.content

### 3.4、Prompt 模板


In [13]:
def build_prompt(prompt_template, **kwargs):
    '''将 Prompt 模板赋值'''
    prompt = prompt_template
    for k, v in kwargs.items():
        if isinstance(v, str):
            val = v
        elif isinstance(v, list) and all(isinstance(elem, str) for elem in v):
            val = '\n'.join(v)
        else:
            val = str(v)
        prompt = prompt.replace(f"__{k.upper()}__", val)
    return prompt

In [20]:
prompt_template = """
你是一个问答机器人。
你的任务是根据下述给定的已知信息回答用户问题。
确保你的回复完全依据下述已知信息。不要编造答案。
如果下述已知信息不足以回答用户的问题，请直接回复"我无法回答您的问题"。

已知信息:
__INFO__

用户问：
__QUERY__

请用中文回答用户问题。
"""

In [19]:
prompt = build_prompt(prompt_template, info="a", query="b", key="c")
print(prompt)


你是一个问答机器人。
你的任务是根据下述给定的已知信息回答用户问题。
确保你的回复完全依据下述已知信息。不要编造答案。
如果下述已知信息不足以回答用户的问题，请直接回复"我无法回答您的问题"。

已知信息:
a

用户问：
b

c

请用中文回答用户问题。



### 3.5、RAG Pipeline 初探


<video src="RAG.mp4" controls="controls" width=800px style="margin-left: 0px"></video>



In [21]:
user_query = "how many parameters does llama 2 have?"

# 1. 检索
search_results = search(user_query, 2)

# 2. 构建 Prompt
prompt = build_prompt(prompt_template, info=search_results, query=user_query)
print("===Prompt===")
print(prompt)

# 3. 调用 LLM
response = get_completion(prompt)

print("===回复===")
print(response)

===Prompt===

你是一个问答机器人。
你的任务是根据下述给定的已知信息回答用户问题。
确保你的回复完全依据下述已知信息。不要编造答案。
如果下述已知信息不足以回答用户的问题，请直接回复"我无法回答您的问题"。

已知信息:
 Llama 2 comes in a range of parameter sizes—7B, 13B, and 70B—as well as pretrained and fine-tuned variations.
 1. Llama 2, an updated version of Llama 1, trained on a new mix of publicly available data. We also increased the size of the pretraining corpus by 40%, doubled the context length of the model, and adopted grouped-query attention (Ainslie et al., 2023). We are releasing variants of Llama 2 with 7B, 13B, and 70B parameters. We have also trained 34B variants, which we report on in this paper but are not releasing.§

用户问：
how many parameters does llama 2 have?

请用中文回答用户问题。

===回复===
Llama 2有7B、13B和70B三种参数大小。


<div class="alert alert-info">
<b>扩展阅读：</b>
<ol>
<ul>Elasticsearch（简称ES）是一个广泛应用的开源搜索引擎: https://www.elastic.co/</ul>
<ul>关于ES的安装、部署等知识，网上可以找到大量资料，例如: https://juejin.cn/post/7104875268166123528</ul>
<ul>关于经典信息检索技术的更多细节，可以参考: https://nlp.stanford.edu/IR-book/information-retrieval-book.html</ul>
</div>


### 3.6、关键字检索的局限性


同一个语义，用词不同，可能导致检索不到有效的结果


In [24]:
user_query="Does llama 2 have a chat version?"
# user_query = "Does llama 2 have a conversational variant?"

search_results = search(user_query, 2)

for res in search_results:
    print(res+"\n")

 2. Llama 2-Chat, a fine-tuned version of Llama 2 that is optimized for dialogue use cases. We release

 Figure 20: Distribution shift for progressive versions of Llama 2-Chat, from SFT models towards RLHF.



## 四、向量检索


### 4.1、文本向量（Text Embeddings）


1. 将文本转成一组浮点数：每个下标 $i$，对应一个维度
2. 整个数组对应一个 $n$ 维空间的一个点，即**文本向量**又叫 Embeddings
3. 向量之间可以计算距离，距离远近对应**语义相似度**大小

<br />
<img src="embeddings.png" style="margin-left: 0px" width=800px>
<br />


### 4.1.1、文本向量是怎么得到的（选）

1. 构建相关（正立）与不相关（负例）的句子对儿样本
2. 训练双塔式模型，让正例间的距离小，负例间的距离大

例如：

<img src="sbert.png" style="margin-left: 0px" width=500px>


<div class="alert alert-info">
<b>扩展阅读：https://www.sbert.net</b>
</div>


### 4.2、向量间的相似度计算


<img src="sim.png" style="margin-left: 0px" width=500px>

余弦相似度取值为-1到1，-1最不相似，1最相似！

In [26]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [27]:
def cos_sim(a, b):
    '''余弦距离 -- 越大越相似'''
    return dot(a, b)/(norm(a)*norm(b))


def l2(a, b):
    '''欧式距离 -- 越小越相似'''
    x = np.asarray(a)-np.asarray(b)
    return norm(x)

In [28]:
def get_embeddings(texts, model="text-embedding-ada-002",dimensions=None):
    '''封装 OpenAI 的 Embedding 模型接口，文档地址 https://platform.openai.com/docs/guides/embeddings/what-are-embeddings'''
    if model == "text-embedding-ada-002":
        dimensions = None
    if dimensions:
        data = client.embeddings.create(input=texts, model=model, dimensions=dimensions).data
    else:
        data = client.embeddings.create(input=texts, model=model).data
    return [x.embedding for x in data]

In [31]:
test_query = ["测试文本"]
vec = get_embeddings(test_query, dimensions=128)[0]
print(vec[:10])
print(len(vec))

[-0.007280634716153145, -0.006147929932922125, -0.010664181783795357, 0.001484171487390995, -0.010678750462830067, 0.029253656044602394, -0.01976952701807022, 0.005444996990263462, -0.01687038503587246, -0.01207733154296875]
1536


In [32]:
# query = "国际争端"

# 且能支持跨语言
query = "global conflicts"

documents = [
    "联合国就苏丹达尔富尔地区大规模暴力事件发出警告",
    "土耳其、芬兰、瑞典与北约代表将继续就瑞典“入约”问题进行谈判",
    "日本岐阜市陆上自卫队射击场内发生枪击事件 3人受伤",
    "国家游泳中心（水立方）：恢复游泳、嬉水乐园等水上项目运营",
    "我国首次在空间站开展舱外辐射生物学暴露实验",
]

query_vec = get_embeddings([query])[0]
doc_vecs = get_embeddings(documents)

print("Cosine distance:")
print(cos_sim(query_vec, query_vec))

for vec in doc_vecs:
    print(cos_sim(query_vec, vec))

print("\nEuclidean distance:")
print(l2(query_vec, query_vec))
for vec in doc_vecs:
    print(l2(query_vec, vec))

Cosine distance:
1.0
0.7622749944010911
0.7563038106493583
0.7426665802579038
0.7079273699608006
0.7254355321045071

Euclidean distance:
0.0
0.6895288502682276
0.6981349637998768
0.7174028746492277
0.764293983363683
0.7410323668625171


<div class="alert alert-warning">
<b>思考：</b>如果我有1000万个文档要去做相似度的计算，该怎么办？
</div>


### 4.3、向量数据库


一个个的比较效率实在过低，我们需要专业的数据库来帮助我们完成这种对于向量的计算的操作。

**向量数据库**，是专门为向量检索设计的中间件。这里我们使用一种开源的向量数据库 chromadb，文档地址 https://docs.trychroma.com/getting-started#1.-install

In [26]:
!pip install chromadb==0.3.29

In [33]:
# 为了演示方便，我们只取两页（第一章）
paragraphs = extract_text_from_pdf(
    "llama2.pdf", 
    page_numbers=[2, 3], 
    min_line_length=10
)

In [34]:
import chromadb
from chromadb.config import Settings


class MyVectorDBConnector:
    def __init__(self, collection_name, embedding_fn):
        chroma_client = chromadb.Client(Settings(allow_reset=True))

        # 为了演示，实际不需要每次 reset()
        chroma_client.reset()

        # 创建一个 collection
        self.collection = chroma_client.get_or_create_collection(name=collection_name)
        self.embedding_fn = embedding_fn

    def add_documents(self, documents):
        '''向 collection 中添加文档与向量'''
        self.collection.add(
            embeddings=self.embedding_fn(documents),  # 每个文档的向量
            documents=documents,  # 文档的原文
            ids=[f"id{i}" for i in range(len(documents))]  # 每个文档的 id
        )

    def search(self, query, top_n):
        '''检索向量数据库'''
        results = self.collection.query(
            query_embeddings=self.embedding_fn([query]),
            n_results=top_n
        )
        return results

In [35]:
# 创建一个向量数据库对象
vector_db = MyVectorDBConnector("demo", get_embeddings)
# 向向量数据库中添加文档
vector_db.add_documents(paragraphs)

In [36]:
user_query = "Llama 2有多少参数"
results = vector_db.search(user_query, 2)

In [37]:
for para in results['documents'][0]:
    print(para+"\n")

 1. Llama 2, an updated version of Llama 1, trained on a new mix of publicly available data. We also increased the size of the pretraining corpus by 40%, doubled the context length of the model, and adopted grouped-query attention (Ainslie et al., 2023). We are releasing variants of Llama 2 with 7B, 13B, and 70B parameters. We have also trained 34B variants, which we report on in this paper but are not releasing.§

 In this work, we develop and release Llama 2, a family of pretrained and fine-tuned LLMs, Llama 2 and Llama 2-Chat, at scales up to 70B parameters. On the series of helpfulness and safety benchmarks we tested, Llama 2-Chat models generally perform better than existing open-source models. They also appear to be on par with some of the closed-source models, at least on the human evaluations we performed (see Figures 1 and 3). We have taken measures to increase the safety of these models, using safety-specific data annotation and tuning, as well as conducting red-teaming and e

<div class="alert alert-success">
<b>澄清几个关键概念：</b><ul>
    <li>向量数据库的意义是快速的检索；</li>
    <li>向量数据库本身不生成向量，向量是由 Embedding 模型产生的；</li>
    <li>向量数据库与传统的关系型数据库是互补的，不是替代关系，在实际应用中根据实际需求经常同时使用。</li>
</ul>
</div>


### 4.3.1、向量数据库服务


Server 端

```sh
chroma run --path /db_path
```

Client 端

```python
import chromadb
chroma_client = chromadb.HttpClient(host='localhost', port=8000)
```


### 4.3.2、主流向量数据库功能对比

<img src="vectordb.png" style="margin-left: 0px" width=600px>


- FAISS: Meta 开源的向量检索引擎 https://github.com/facebookresearch/faiss
- Pinecone: 商用向量数据库，只有云服务 https://www.pinecone.io/
- Milvus: 开源向量数据库，同时有云服务 https://milvus.io/
- Weaviate: 开源向量数据库，同时有云服务 https://weaviate.io/
- Qdrant: 开源向量数据库，同时有云服务 https://qdrant.tech/
- PGVector: Postgres 的开源向量检索引擎 https://github.com/pgvector/pgvector
- RediSearch: Redis 的开源向量检索引擎 https://github.com/RediSearch/RediSearch
- ElasticSearch 也支持向量检索 https://www.elastic.co/enterprise-search/vector-search


### 4.4、基于向量检索的 RAG


In [38]:
class RAG_Bot:
    def __init__(self, vector_db, llm_api, n_results=2):
        self.vector_db = vector_db
        self.llm_api = llm_api
        self.n_results = n_results

    def chat(self, user_query):
        # 1. 检索
        search_results = self.vector_db.search(user_query, self.n_results)

        # 2. 构建 Prompt
        prompt = build_prompt(
            prompt_template, info=search_results['documents'][0], query=user_query)

        # 3. 调用 LLM
        response = self.llm_api(prompt)
        return response

In [39]:
# 创建一个RAG机器人
bot = RAG_Bot(
    vector_db,
    llm_api=get_completion
)

user_query = "llama 2有对话版吗？"

response = bot.chat(user_query)

print(response)

已知信息中提到了"Llama 2-Chat"，这是一个专为对话使用情况优化的Llama 2的精调版本。因此，是的，Llama 2有对话版。


### 4.5、如果想要换个国产模型


In [34]:
import json
import requests
import os

# 通过鉴权接口获取 access token
def get_access_token():
    """
    使用 AK，SK 生成鉴权签名（Access Token）
    :return: access_token，或是None(如果错误)
    """
    url = "https://aip.baidubce.com/oauth/2.0/token"
    params = {
        "grant_type": "client_credentials",
        "client_id": os.getenv('ERNIE_CLIENT_ID'),
        "client_secret": os.getenv('ERNIE_CLIENT_SECRET')
    }

    return str(requests.post(url, params=params).json().get("access_token"))

# 调用文心千帆 调用 BGE Embedding 接口
def get_embeddings_bge(prompts):
    url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/embeddings/bge_large_en?access_token=" + get_access_token()
    payload = json.dumps({
        "input": prompts
    })
    headers = {'Content-Type': 'application/json'}

    response = requests.request(
        "POST", url, headers=headers, data=payload).json()
    data = response["data"]
    return [x["embedding"] for x in data]


# 调用文心4.0对话接口
def get_completion_ernie(prompt):

    url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/completions_pro?access_token=" + get_access_token()
    payload = json.dumps({
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]
    })

    headers = {'Content-Type': 'application/json'}

    response = requests.request(
        "POST", url, headers=headers, data=payload).json()

    return response["result"]

In [ ]:
# 创建一个向量数据库对象
new_vector_db = MyVectorDBConnector(
    "demo_ernie",
    embedding_fn=get_embeddings_bge
)
# 向向量数据库中添加文档
new_vector_db.add_documents(paragraphs)

# 创建一个RAG机器人
new_bot = RAG_Bot(
    new_vector_db,
    llm_api=get_completion_ernie
)

In [ ]:
user_query = "how many parameters does llama 2 have?"

response = new_bot.chat(user_query)

print(response)

### 4.6、OpenAI 新发布的两个 Embedding 模型

2024年1月25日，OpenAI 新发布了两个 Embedding 模型

- text-embedding-3-large
- text-embedding-3-small

其最大特点是，支持自定义的缩短向量维度，从而在几乎不影响最终效果的情况下降低向量检索与相似度计算的复杂度。

通俗的说：**越大越准、越小越快。** 官方公布的评测结果:

<img src="mteb.png" style="margin-left: 0px" width=600px>

注：[MTEB](https://huggingface.co/blog/mteb) 是一个大规模多任务的 Embedding 模型公开评测集

In [37]:
model = "text-embedding-3-large"
dimensions = 128

query = "国际争端"

# 且能支持跨语言
# query = "global conflicts"

documents = [
    "联合国就苏丹达尔富尔地区大规模暴力事件发出警告",
    "土耳其、芬兰、瑞典与北约代表将继续就瑞典“入约”问题进行谈判",
    "日本岐阜市陆上自卫队射击场内发生枪击事件 3人受伤",
    "国家游泳中心（水立方）：恢复游泳、嬉水乐园等水上项目运营",
    "我国首次在空间站开展舱外辐射生物学暴露实验",
]

query_vec = get_embeddings([query],model=model,dimensions=dimensions)[0]
doc_vecs = get_embeddings(documents,model=model,dimensions=dimensions)

print("Dim: {}".format(len(query_vec)))

print("Cosine distance:")
for vec in doc_vecs:
    print(cos_sim(query_vec, vec))

print("\nEuclidean distance:")
for vec in doc_vecs:
    print(l2(query_vec, vec))

Dim: 128
Cosine distance:
0.2865773001182426
0.41830986590456204
0.21462566338499087
0.15146227929798226
0.17059296471763005

Euclidean distance:
1.1945063968652392
1.0786010898034128
1.2532951931301017
1.3027185279508469
1.287949624282259


<div class="alert alert-info">
<b>扩展阅读：这种可变长度的 Embedding 技术背后的原理叫做 <a href="https://arxiv.org/abs/2205.13147">Matryoshka Representation Learning</a> </b>
</div>


## 总结一下 RAG


![](./rag-flow.jpg)


如图所示，RAG主要由两个部分构成：
- **建立索引**：首先要清洗和提取原始数据，将 PDF、Docx等不同格式的文件解析为纯文本数据；然后将文本数据分割成更小的片段（chunk）；最后将这些片段经过嵌入模型转换成向量数据（此过程叫做embedding），并将原始语料块和嵌入向量以键值对形式存储到向量数据库中，以便进行后续快速且频繁的搜索。这就是建立索引的过程。
    - 文档加载，并按一定条件**切割**成片段
    - 将切割后的文档转化为 embedding
    - 把 embedding 存储到 embeddingStore 中
- **检索生成**：系统会获取到用户输入，随后计算出用户的问题与向量数据库中的文档块之间的相似度，选择相似度最高的K个文档块（K值可以自己设置）作为回答当前问题的知识。知识与问题会合并到提示词模板中提交给大模型，大模型给出回复。这就是检索生成的过程。
    - 把用户 Query 转化为 embedding，queryEmbedding
    - 检索最相似的几个文档，找到最相似的 K 个
    - 取回 K 个最相似的文档文本
    - 把文本发送给大模型“包装”，生成最终返回给用户的文本

## 作业：
- 1. 独立完成 RAG 流程的构建代码
- 2. 输入你自己的文档，让RAG来回答，看效果如何？